In [2]:
import torch
from data_handler import create_dataset, save_json, load_json, load_encoded_data
from model import TravelDocClassifier
from cuda_utils import clear_cache, print_cuda_memory_info

c:\Users\wei\Desktop\programs\information-retrieval\ir_final_doc_classifier\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
clear_cache()
print_cuda_memory_info()

Device: NVIDIA GeForce RTX 4070 SUPER
Total Memory: 12.88 GB
Reserved Memory: 0.52 GB
Allocated Memory: 0.42 GB
Free Memory: 0.10 GB


In [10]:
encoded_data = load_encoded_data('encoded_data/all_512')
filtered_data = load_json('filtered_data/all.json')

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier = TravelDocClassifier(device=device, params_path='parameters/20240603_95_model_parameters.pth')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
test_dataset = create_dataset(encoded_data)

pred_labels = classifier.predict_all(test_dataset)

travel_related = []
non_travel_related = []

for i, pred_label in enumerate(pred_labels):
    if pred_label == 1:
        travel_related.append(filtered_data[i])
    else:
        non_travel_related.append(filtered_data[i])

len(travel_related), len(non_travel_related)

(39085, 14426)

In [13]:
save_json(travel_related, 'results/travel_related_predictions.json')
save_json(non_travel_related, 'results/non_travel_related_predictions.json')

In [10]:
import re

travel_related_titles = ['遊記', '心得', '問題', '食記', '徵伴', '分享', '住宿', '新聞', '黑特', '旅遊', '玩樂', '請益', '美食', '抱怨', '推薦']

# 徵求 揪團 廣宣 活動
non_travel_related_titles = ['交易', '廣告', '情報', '問卷', '資訊', '徵求', '公告', '講座', '揪團', '贈送', '閒聊', '轉讓', '廣宣', '活動', '已售出', '售出', '票券', '請問', '轉錄', '買賣', '徵']

travel_pattern = re.compile(r'\[' + '|'.join(travel_related_titles) + r'\]')
non_travel_pattern = re.compile(r'\[' + '|'.join(non_travel_related_titles) + r'\]')

In [21]:
predictions_non_travel_data  = load_json('results/non_travel_related_predictions.json')
predictions_travel_data  = load_json('results/travel_related_predictions.json')

misclassified_data = {
    'predicted_travel_but_non_travel': [],
    'predicted_non_travel_but_travel': []
}

# 找出預測為旅遊但實際上不是旅遊的資料
for data in predictions_travel_data:
  title = data['title']
  if re.search(non_travel_pattern, title):
    misclassified_data['predicted_travel_but_non_travel'].append(data)

# 找出預測為非旅遊但實際上是旅遊的資料
for data in predictions_non_travel_data:
  title = data['title']
  if re.search(travel_pattern, title):
    misclassified_data['predicted_non_travel_but_travel'].append(data)

In [22]:
len(misclassified_data['predicted_travel_but_non_travel'])

15149

In [23]:
len(misclassified_data['predicted_non_travel_but_travel'])

11703

In [28]:
import random

def random_sample(data, sample_size=5):
  return random.sample(data, min(len(data), sample_size))


# 打印隨機抽取的資料
print("預測為旅遊但實際上不是旅遊的資料:")
for idx, item in enumerate(random_sample(misclassified_data['predicted_travel_but_non_travel'])):
  print(f'caase: {idx + 1}')
  print(item)

print("\n預測為非旅遊但實際上是旅遊的資料:")
for idx, item in enumerate(random_sample(misclassified_data['predicted_non_travel_but_travel'])):
  print(f'caase: {idx + 1}')
  print(item)

預測為旅遊但實際上不是旅遊的資料:
caase: 1
{'title': '[交易] 收購易遊網旅券3-4萬', 'author': 'tclin076', 'time': {'year': '2024', 'month': 2, 'date': '25', 'hour': '16', 'minute': '02'}, 'content': '交易ID/交易者稱呼 ：tclin076 交易物品簡介或網址： 收購易遊網旅券3-4萬 聯絡方式：站內信 注意事項及補充說明：台北、台中可面交 --'}
caase: 2
{'title': '[廣告] 納米比亞紅人沙漠、辛巴威13天', 'author': 'hsuans', 'time': {'year': '2017', 'month': 9, 'date': '21', 'hour': '10', 'minute': '27'}, 'content': '※ [本文轉錄自 Tour-Agency 看板 #1PmoDnL7 ] 作者: hsuans (大豆) 看板: Tour-Agency 標題: [情報] 納米比亞紅人沙漠、辛巴威13天 時間: Thu Sep 21 10:26:54 2017 ◎ 請注意旅行社廠商自介過後保證為正常經營廠商絕非靠行，才可發情報文。 \u3000◎ 情報文除了行程資訊與報價外，請詳述給 PTT鄉民 的實質優惠，謝謝。（必要） --< 以下為情報文 >-- 納米比亞紅人沙漠、辛巴威維多利亞瀑布13日(觀景飛機雙飛) https://goo.gl/g6UDzH 有一群人，天生就是「紅人」 你以為世界都去過了嗎 世界上最大的瀑布--維多利亞瀑布 世界最不值錢的辛巴威紙鈔 還有世界最有名的紅人沙漠 要你親自體驗 10/20出發，本團已額滿，缺一單女，現有5000折價。 -- LINE ID:HSUANS 大豆 名\u3000\u3000稱:理想旅運社股份有限公司-綜合旅行社 電\u3000\u3000話:(02)7729-1000 分機225 註冊編號：交觀綜2118．品保北100號 業務專員: 潘宣吾 LINE: hsuans PTT 使用記錄: 登入次數 5372 次 / 文章 8712 篇 -- ※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 59.124.211.133 ※ 文章網